In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers.convolutional import Convolution1D, MaxPooling1D
from keras.layers.normalization import BatchNormalization

# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "ann"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

In [ ]:
from sklearn.metrics import hamming_loss, accuracy_score 

def model_report_multilabel(model_predict, X_train, y_train, X_valid, y_valid):
    def report_dataset(X, y_true, title):
        y_pred = model_predict(X)
        print(title + ' accuracy (exatch match):', accuracy_score(y_true, y_pred))
        print(title + ' hamming score (non-exatch match):', 1 - hamming_loss(y_true, y_pred))

    report_dataset(X_train, y_train, 'training')
    report_dataset(X_valid, y_valid, 'validation')

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

df = pd.read_csv('samples_nylonGuitar_1024_Mm7_R03.csv')

X = np.array(df.iloc[:,:-1], dtype=np.float)
y = np.array(df.iloc[:,-1], dtype=np.float)

In [ ]:
import os

processedData_path = "preprocessedSamples.data"

if os.path.isfile(processedData_path): #if already preprocessed
    df_new = pd.read_pickle(processedData_path)
else:
    for i in range(len(X)):
        sample = np.array(X[i], dtype=np.float)
        sample = sample*np.hamming(1024)
        sample = np.abs(np.fft.rfft(sample))[1:]
        sample = np.append(sample, y[i])
        try:
            df_new = np.vstack([df_new, sample])
        except:
            df_new = np.array(sample, dtype=np.float)
        if i % 200 == 0:
            print(i)
            df_new[i]
    
    df_new = pd.DataFrame(df_new)
    
    df_new.to_pickle(processedData_path)

In [ ]:
import matplotlib.pyplot as plt

npArrayDF = np.array(df_new.iloc[:,:], dtype=np.float) #shuffle randomly all samples

print(npArrayDF[0,-1])

for i in range(len(npArrayDF)):
    npArrayDF[i,-1] = (npArrayDF[i, -1]) - 1

print(npArrayDF[0,-1])
    
np.random.shuffle(npArrayDF)

X_train = np.array(npArrayDF[:-2000,:-1], dtype=np.float)
y_train = np.array(npArrayDF[:-2000,-1], dtype=np.float)

X_valid = np.array(npArrayDF[-2000:-1000,:-1], dtype=np.float)
y_valid = np.array(npArrayDF[-2000:-1000,-1], dtype=np.float)

X_test = np.array(npArrayDF[-1000:,:-1], dtype=np.float)
y_test = np.array(npArrayDF[-1000:,-1], dtype=np.float)
print(y_test[1])

plt.plot(X_train[1])
plt.show()

In [ ]:
model = Sequential()

model.add(Convolution1D(32, (3,3), input_shape=(X_train.shape[1], 1)))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Convolution1D(32, (3,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling1D(2, 2))

model.add(Flatten())

model.add(Dense(512))
model.add(Activation('relu'))

model.add(Dense(512))
model.add(Activation('relu'))

model.add(Dense(256))
model.add(Activation('sigmoid'))

print('number of parameters:', model.count_params())
print('weights', [w.shape for w in model.get_weights()])

model.compile(class_mode='binary', loss='binary_crossentropy', optimizer='adam')

In [8]:
hist = model.fit(X_train, y_train, nb_epoch=100)

C:\Users\Denis\AppData\Local\conda\conda\envs\tensorflow_env_gpu\lib\site-packages\keras\models.py:844: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


ValueError: Error when checking input: expected conv1d_1_input to have 3 dimensions, but got array with shape (12377, 512)

In [ ]:
model_report_multilabel(model.predict_classes, X_train, y_train, X_valid, y_valid)